In [57]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

In [77]:
#read csv file
city_data = pd.read_csv("Output_data/cities.csv" ,index_col=0)
city_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,viedma,0,AR,1666407485,70,-40.8135,-62.9967,39.16,6.44
1,klaksvik,80,FO,1666407485,78,62.2266,-6.5890,41.92,21.97
2,jamestown,0,US,1666407463,59,42.0970,-79.2353,55.71,13.02
3,new norfolk,100,AU,1666407486,89,-42.7826,147.0587,59.90,0.76
4,punta arenas,75,CL,1666407486,61,-53.1500,-70.9167,44.71,21.85


# Humidity Heatmap

In [59]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)


In [60]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Lat", "Lng"]].astype(float)
weights= city_data["Humidity"]

# Create HeatMap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights = weights,
                              dissipating=False, max_intensity=100,
                              point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [68]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Zero cloudiness
new_df = city_data.loc[(city_data["Max Temp"]>=70) & (city_data["Max Temp"]<80)]
new_df = new_df.loc[(new_df["Wind Speed"]>10) & (new_df["Cloudiness"]==0)]
filtered_df = new_df.reset_index(drop=True)
#remove rows with null values
filtered_df.dropna()
#ctreating new dataframe with few columns and new column
hotel_df = filtered_df[["City","Country","Cloudiness","Lat","Lng","Max Temp","Wind Speed"]]
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\13135\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,City,Country,Cloudiness,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,atuona,PF,0,-9.8000,-139.0333,77.05,14.12,
1,carnarvon,AU,0,-24.8667,113.6333,72.90,13.51,
2,san patricio,US,0,28.0170,-97.5169,73.89,13.33,
3,enid,US,0,36.3956,-97.8784,76.30,10.36,
4,buraydah,SA,0,26.3260,43.9750,71.10,10.45,
5,haysville,US,0,37.5645,-97.3523,78.48,17.27,
6,mahajanga,MG,0,-15.7167,46.3167,70.77,10.33,


In [76]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key}

#looping through the dataframe
for index,row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
     # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
     # convert to json
    response = hotel_name.json()
    try:
        hotel_df.loc[index,"Hotel Name"]=response['results'][0]['name']        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#     pprint(response)

In [70]:
# display the dataframe with new values
hotel_df

,City,Country,Cloudiness,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,atuona,PF,0,-9.8000,-139.0333,77.05,14.12,Hôtel Hiva Oa Hanakee Lodge
1,carnarvon,AU,0,-24.8667,113.6333,72.90,13.51,Hospitality Carnarvon
2,san patricio,US,0,28.0170,-97.5169,73.89,13.33,Americas Best Value Inn Sinton
3,enid,US,0,36.3956,-97.8784,76.30,10.36,Hampton Inn & Suites Enid
4,buraydah,SA,0,26.3260,43.9750,71.10,10.45,Rollins Hotel
5,haysville,US,0,37.5645,-97.3523,78.48,17.27,Sleep Inn & Suites
6,mahajanga,MG,0,-15.7167,46.3167,70.77,10.33,Baobab Tree Hotel & Spa


In [78]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

# Create HeatMap

fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig



Figure(layout=FigureLayout(height='420px'))

In [79]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))